In [15]:
import torch
from datasets import load_dataset
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

In [8]:
dataset = load_dataset("deepmind/code_contests", split="train", streaming=True) #cf_tags
for it in dataset:
    print(it['description'])
    print(it['cf_tags'])
    break

Problem description.
Vipul is a hardworking super-hero who maintains the bracket ratio of all the strings in the world. Recently he indulged himself in saving the string population so much that he lost his ability for checking brackets (luckily, not permanently ).Being his super-hero friend help him in his time of hardship. 

Input

The first line of the input contains an integer T denoting the number of test cases. The description of T test cases follows.
The first line of each test case contains a single string S denoting the string to be checked.


Output

For each test case, output a single line printing "YES" or "NO" (without " " and in uppercase only) , denoting if the brackets in the given string is balanced or not .


Constraints

1 ≤ T ≤ 10
1 ≤ length of S ≤ 60


Example
Input:
3
((()))
(())()
()(()

Output:
YES
YES
NO

 

Explanation
Example is self-explanatory.
[]


In [ ]:
def getDataset(split):
    dataset = load_dataset("deepmind/code_contest", split=split)   
    tags = []
    descriptions = [] 
    for x in dataset:
        if(len(dataset['cf_tags'] == 0)) continue
        tags.append(dataset['cf_tags'])
        descriptions.append(dataset['description'])
    return tags, descriptions

In [9]:
def getSetFromTags(tag_list):
    my_set = set()
    for sub_list in my_list:
        for element in sub_list:
            my_set.add(element)

In [10]:
def getOneHot(tag_list, tag_set):
    index_dict = {}
    for i, element in enumerate(tag_set):
        index_dict[element] = i

    one_hot_list = torch.zeros((len(tag_list),len(tag_set)))
    for i, sub_list in enumerate(tag_list):
        for element in sub_list:
            idx = index_dict[element]
            one_hot_list[i][idx]=1
    
    return one_hot_list, index_dict
    


In [ ]:
tr_x, tr_tags = getDataset('train')
tags_set = getSetFromTags(tr_tags)
tr_y, tags_dict = getOneHot(tr_tags, tags_set)

In [13]:
class TagsDataset(Dataset):
    def __init__(self, descriptions, labels, one_hot):
        self.descriptions = descriptions
        self.labels = labels
        self.Y = one_hot
    
    def __len__(self):
        return len(self.descriptions)
    
    def __getitem__(self, index):
        x = (torch.Tensor(self.data[index]), torch.Tensor(self.labels[index]))
        y = torch.Tensor(self.Y[index])
        
        return x, y

In [ ]:
dataset = TagsDataset(tr_x, tr_tags, tr_y)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

In [16]:
class LEAM(nn.Module):
    def __init__(self, embedding_dim, num_classes):
        super(LEAM, self).__init__()
        self.embedding_dim = embedding_dim
        self.num_classes = num_classes
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.attention = nn.Linear(embedding_dim, num_classes, bias=False)
        self.label_embeddings = nn.Embedding(num_classes, embedding_dim)

    def forward(self, x, labels):
        # x: input text sequence, shape (batch_size, seq_length)
        # labels: multi-hot encoded label vectors, shape (batch_size, num_classes)

        # Embed the input text sequence
        x_embedded = self.embedding(x)  # shape (batch_size, seq_length, embedding_dim)

        # Compute attention weights for each class
        attention_weights = self.attention(x_embedded)  # shape (batch_size, seq_length, num_classes)

        # Apply softmax activation to get attention probabilities
        attention_probs = F.softmax(attention_weights, dim=1)  # shape (batch_size, seq_length, num_classes)

        # Weight the label embeddings by the attention probabilities
        label_weights = torch.bmm(attention_probs.transpose(1, 2), self.label_embeddings(labels))  # shape (batch_size, embedding_dim)

        # Compute the final prediction scores
        prediction = torch.sum(x_embedded * attention_probs, dim=1) + label_weights  # shape (batch_size, embedding_dim)
        prediction = F.sigmoid(prediction)  # shape (batch_size, num_classes)

        return prediction